In [1]:
# Author: Caleb Woy
!python --version
import pandas as pd # data processing
from math import comb # combinatorics

Python 3.8.2


# Importing data

In [2]:
path_to_data = "C:/Users/woybo/Desktop/jupyter-Nbooks/DM-Associations"

# Loading the training and test data sets into pandas
data = pd.read_csv(path_to_data + "/assoc1.csv", header=0)
data = data.drop(columns=["TxId"])

In [3]:
data.head()

,Bacon,Butter,Cereal,Eggs,Flour,Milk
0,1,0,1,0,0,0
1,0,1,0,0,1,1
2,0,0,1,0,0,1
3,0,1,0,1,1,1
4,1,0,0,1,0,1


I've loaded the transaction data into a pandas dataframe from a csv. I dropped the transaction id column because pandas default indexing is sufficient to uniqely identify a transaction.

# Calculating basic statistics

#### A. Total number of itemsets from d items: $2^d$

In [4]:
print(f'Total # of possible itemsets: {2 ** 6}')

Total # of possible itemsets: 64


#### B. Total number of rules: $\sum_{k=1}^{d-1}[{d \choose k} x \sum_{j=1}^{d-k}{d - k \choose j}]$

In [5]:
total = 0
d = 6
for k in range(1, 6):
    sub_total = 0
    for j in range(1, d - k + 1):
        sub_total += comb(d - k, j)
    total += comb(d, k) * sub_total
print(f'Total # of possible rules: {total}')

Total # of possible rules: 602


#### C. Observations:
The number of itemsets is bounded by $\Theta(2^d)$ where d is the number of unique itemsets.<br/>
The number of rules is even higher than that.<br/>
A brute force rule generation scheme is definitely not possible with higher dimension data sets.

# Generating frequent itemsets

#### A. Calculating the number of actual itemsets in our data:

In [6]:
print(data.columns.values)

['Bacon' 'Butter' 'Cereal' 'Eggs' 'Flour' 'Milk']


In [7]:
from itertools import combinations 

itemsets = {}
items = data.columns.values.tolist()
combos = []

for i in range(1, len(items) + 1):
    for j in list(combinations(items, i)):
        combos.append(j)
for item_tuple in combos:
    counts = data.groupby([j for j in item_tuple]).size()
    for k in range(len(item_tuple)):
        counts = counts[1]
    itemsets[item_tuple] = counts
    
print(f'Number of itemsets in our data: {len(itemsets)}')

Number of itemsets in our data: 63


#### B. Finding the fequent itemsets for MinSup = 2, 3, 4:

In [8]:
def Min_sup_print(min_sup, itemsets):
    print(f'Min support: {min_sup}')
    counter = 0
    for s in itemsets:
        if itemsets[s] >= min_sup:
            print(f' {s}: support {itemsets[s]}')
            counter += 1
    print(f'Total itemsets: {counter}')
    print()
        
Min_sup_print(2, itemsets)
Min_sup_print(3, itemsets)
Min_sup_print(4, itemsets)

Min support: 2
 ('Bacon',): support 20
 ('Butter',): support 24
 ('Cereal',): support 20
 ('Eggs',): support 17
 ('Flour',): support 21
 ('Milk',): support 29
 ('Bacon', 'Butter'): support 2
 ('Bacon', 'Cereal'): support 8
 ('Bacon', 'Eggs'): support 8
 ('Bacon', 'Flour'): support 4
 ('Bacon', 'Milk'): support 5
 ('Butter', 'Cereal'): support 8
 ('Butter', 'Eggs'): support 9
 ('Butter', 'Flour'): support 18
 ('Butter', 'Milk'): support 17
 ('Cereal', 'Eggs'): support 3
 ('Cereal', 'Flour'): support 5
 ('Cereal', 'Milk'): support 11
 ('Eggs', 'Flour'): support 11
 ('Eggs', 'Milk'): support 13
 ('Flour', 'Milk'): support 13
 ('Bacon', 'Butter', 'Eggs'): support 2
 ('Bacon', 'Butter', 'Flour'): support 2
 ('Bacon', 'Cereal', 'Milk'): support 2
 ('Bacon', 'Eggs', 'Flour'): support 4
 ('Bacon', 'Eggs', 'Milk'): support 4
 ('Butter', 'Cereal', 'Eggs'): support 2
 ('Butter', 'Cereal', 'Flour'): support 4
 ('Butter', 'Cereal', 'Milk'): support 5
 ('Butter', 'Eggs', 'Flour'): support 9
 ('Butte

#### C. For MinSup = 3, prune the ItemSets via the Apriori algorithm.

In [9]:
def Apriori(df, min_sup, verbose=True):
    result, f, c, all_items = {}, {}, {}, df.columns.values
    for item in all_items:
        key = (item,)
        f[key] = (df[item] == 1).sum()
    for key, value in list(f.items()):
        if value < min_sup:
            del f[key]
    result.update(f)
    if f:
        all_items = [i[0] for i in f]
        for i in range(len(all_items)):
            for j in range(i + 1, len(all_items)):
                item_tuple = (all_items[i], all_items[j])
                counts = df.groupby([i for i in item_tuple]).size()
                for k in range(len(item_tuple)):
                    counts = counts[1]
                c[item_tuple] = counts
        f, c = c, {}
        for key, value in list(f.items()):
            if value < min_sup:
                del f[key]
        result.update(f)
        while f:
            all_items = [i for i in f]
            for i in range(len(all_items)):
                for j in range(i + 1, len(all_items)):
                    same = True
                    for k in range(len(all_items[i]) - 1):
                        if not same or not all_items[i][k] == all_items[j][k]:
                            same = False
                    item_tuple = all_items[i] + (all_items[j][len(all_items[j]) - 1], )
                    if same:
                        counts = df.groupby([i for i in item_tuple]).size()
                        for k in range(len(item_tuple)):
                            counts = counts[1]
                        c[item_tuple] = counts
            f, c = c, {}
            for key, value in list(f.items()):
                if value < min_sup:
                    del f[key]
            result.update(f)
    if verbose:
        print(f'Min support: {min_sup}, Total itemsets: {len(result)}, Total pruned: {63 - len(result)}')
        for key, value in list(result.items()):
            print(f' {key}, support: {value}')    
    return result
    
x = Apriori(data, 3)

Min support: 3, Total itemsets: 31, Total pruned: 32
 ('Bacon',), support: 20
 ('Butter',), support: 24
 ('Cereal',), support: 20
 ('Eggs',), support: 17
 ('Flour',), support: 21
 ('Milk',), support: 29
 ('Bacon', 'Cereal'), support: 8
 ('Bacon', 'Eggs'), support: 8
 ('Bacon', 'Flour'), support: 4
 ('Bacon', 'Milk'), support: 5
 ('Butter', 'Cereal'), support: 8
 ('Butter', 'Eggs'), support: 9
 ('Butter', 'Flour'), support: 18
 ('Butter', 'Milk'), support: 17
 ('Cereal', 'Eggs'), support: 3
 ('Cereal', 'Flour'), support: 5
 ('Cereal', 'Milk'), support: 11
 ('Eggs', 'Flour'), support: 11
 ('Eggs', 'Milk'), support: 13
 ('Flour', 'Milk'), support: 13
 ('Bacon', 'Eggs', 'Flour'), support: 4
 ('Bacon', 'Eggs', 'Milk'), support: 4
 ('Butter', 'Cereal', 'Flour'), support: 4
 ('Butter', 'Cereal', 'Milk'), support: 5
 ('Butter', 'Eggs', 'Flour'), support: 9
 ('Butter', 'Eggs', 'Milk'), support: 8
 ('Butter', 'Flour', 'Milk'), support: 12
 ('Cereal', 'Eggs', 'Milk'), support: 3
 ('Cereal', 'Flou

#### D. Find the set of Maximal Frequent ItemSets.

In [10]:
def Max_freq_sets(df, min_sup):
    freq_sets = Apriori(df, min_sup, False)
    sets_by_size = {i: [] for i in range(1, len(df.columns.values) + 1)}
    max_sets = set()
    for i in freq_sets:
        sets_by_size[len(i)].append(i) 
    for i in freq_sets:
        size = len(i)
        is_max = True
        for j in range(size + 1, len(df.columns.values)):
            if not is_max:
                break
            for k in sets_by_size[j]:
                if set(i).issubset(k):
                    is_max = False
        if is_max:
            max_sets.add(i)
    print(f'Maximal frequent sets with support: {min_sup}, Total itemsets: {len(max_sets)}')
    for i in max_sets:
        print(f' {i}, support: {freq_sets[i]}')
            
Max_freq_sets(data, 3)

Maximal frequent sets with support: 3, Total itemsets: 8
 ('Cereal', 'Flour', 'Milk'), support: 3
 ('Cereal', 'Eggs', 'Milk'), support: 3
 ('Butter', 'Cereal', 'Milk'), support: 5
 ('Bacon', 'Eggs', 'Flour'), support: 4
 ('Bacon', 'Cereal'), support: 8
 ('Butter', 'Eggs', 'Flour', 'Milk'), support: 8
 ('Bacon', 'Eggs', 'Milk'), support: 4
 ('Butter', 'Cereal', 'Flour'), support: 4


#### E. Find the set of Closed ItemSets and the set of Closed Frequent ItemSets.

In [11]:
def Closed_sets(df):
    item_sets = Apriori(df, 1, False)
    sets_by_support = {i: [] for i in range(0, len(df))}
    closed_sets = set()
    for key, value in list(item_sets.items()):
        sets_by_support[value].append(key)
    for i in item_sets:
        supp = item_sets[i]
        is_closed = True
        for j in sets_by_support[supp]:
            if not j == i and set(i).issubset(j):
                is_closed = False
        if is_closed:
            closed_sets.add(i)
        
    print(f'Total Closed item sets: {len(closed_sets)}')
    for i in closed_sets:
        print(f' {i}, support: {item_sets[i]}')
            
Closed_sets(data)

Total Closed item sets: 30
 ('Cereal', 'Flour', 'Milk'), support: 3
 ('Bacon', 'Eggs', 'Milk'), support: 4
 ('Bacon', 'Eggs'), support: 8
 ('Bacon', 'Butter', 'Cereal', 'Eggs', 'Flour', 'Milk'), support: 1
 ('Cereal', 'Milk'), support: 11
 ('Butter', 'Cereal', 'Flour'), support: 4
 ('Eggs', 'Flour'), support: 11
 ('Cereal', 'Eggs', 'Milk'), support: 3
 ('Butter', 'Cereal', 'Milk'), support: 5
 ('Bacon',), support: 20
 ('Bacon', 'Cereal'), support: 8
 ('Butter', 'Flour', 'Milk'), support: 12
 ('Butter', 'Eggs', 'Flour', 'Milk'), support: 8
 ('Eggs',), support: 17
 ('Cereal', 'Flour'), support: 5
 ('Butter', 'Eggs', 'Flour'), support: 9
 ('Butter', 'Cereal', 'Eggs', 'Flour', 'Milk'), support: 2
 ('Milk',), support: 29
 ('Bacon', 'Cereal', 'Milk'), support: 2
 ('Cereal',), support: 20
 ('Flour', 'Milk'), support: 13
 ('Butter', 'Milk'), support: 17
 ('Butter', 'Cereal'), support: 8
 ('Butter',), support: 24
 ('Bacon', 'Milk'), support: 5
 ('Bacon', 'Butter', 'Eggs', 'Flour'), support: 2
 

In [12]:
def Closed_freq_sets(df, min_sup):
    item_sets = Apriori(df, min_sup, False)
    sets_by_support = {i: [] for i in range(0, len(df))}
    closed_sets = set()
    for key, value in list(item_sets.items()):
        sets_by_support[value].append(key)
    for i in item_sets:
        supp = item_sets[i]
        is_closed = True
        for j in sets_by_support[supp]:
            if not j == i and set(i).issubset(j):
                is_closed = False
        if is_closed:
            closed_sets.add(i)
        
    print(f'Total Closed item sets: {len(closed_sets)}')
    for i in closed_sets:
        print(f' {i}, support: {item_sets[i]}')
            
Closed_freq_sets(data, 3)

Total Closed item sets: 26
 ('Cereal', 'Flour', 'Milk'), support: 3
 ('Bacon', 'Eggs'), support: 8
 ('Cereal', 'Milk'), support: 11
 ('Butter', 'Cereal', 'Flour'), support: 4
 ('Eggs', 'Flour'), support: 11
 ('Cereal', 'Eggs', 'Milk'), support: 3
 ('Butter', 'Cereal', 'Milk'), support: 5
 ('Bacon',), support: 20
 ('Bacon', 'Cereal'), support: 8
 ('Butter', 'Flour', 'Milk'), support: 12
 ('Butter', 'Eggs', 'Flour', 'Milk'), support: 8
 ('Eggs',), support: 17
 ('Cereal', 'Flour'), support: 5
 ('Butter', 'Eggs', 'Flour'), support: 9
 ('Milk',), support: 29
 ('Cereal',), support: 20
 ('Flour', 'Milk'), support: 13
 ('Butter', 'Milk'), support: 17
 ('Butter', 'Cereal'), support: 8
 ('Butter',), support: 24
 ('Bacon', 'Milk'), support: 5
 ('Eggs', 'Milk'), support: 13
 ('Bacon', 'Eggs', 'Flour'), support: 4
 ('Flour',), support: 21
 ('Bacon', 'Eggs', 'Milk'), support: 4
 ('Butter', 'Flour'), support: 18


#### F. What do you observe or conclude from this activity? Why is this relevant and/or important?

Item set sizes ranked from largest to smallest:<br/>
1. Frequent item sets
2. Frequent closed item sets
3. Frequent maximal item sets

Finding these types of item sets is important for generating rule sets which fit certain support criteria.

# Generating the Rules

#### A. Calculate the total number of potential Rules that exist in the data.

In [13]:
all_item_sets = Apriori(data, 1, False)
total_rules = 0
for i in all_item_sets:
    total_rules += (2 ** len(i)) - 2
print(f'Total number of rules in our data: {total_rules}')

Total number of rules in our data: 602


#### B. For MinConf = 3, for the ItemSet X = {Butter,Flour,Milk}, prune the rules via the antimonotone property of rules.
#### How many rules were pruned by this process?

In [15]:
def Generate_rules(support_counts, itemset, min_conf, verbose = True):
    result, f, c = {}, {}, {}
    for item in itemset:
        lhs, rhs = tuple(i for i in itemset if i != item), (item,)
        key = (lhs, rhs)
        if lhs:
            f[key] = support_counts[itemset] / support_counts[lhs]
    for key, value in list(f.items()):
        if value < min_conf:
            del f[key]
    result.update(f)
    if f:
        all_items = [i[1][0] for i in f]
        for i in range(len(all_items)):
            for j in range(i + 1, len(all_items)):
                rhs = (all_items[i], all_items[j])
                lhs = tuple(i for i in itemset if i not in rhs)
                key = (lhs, rhs)
                if lhs:
                    c[key] = support_counts[itemset] / support_counts[lhs]
        f, c = c, {}
        for key, value in list(f.items()):
            if value < min_conf:
                del f[key]
        result.update(f)
        while f:
            all_items = [i[1] for i in f]
            for i in range(len(all_items)):
                for j in range(i + 1, len(all_items)):
                    same = True
                    for k in range(len(all_items[i]) - 1):
                        if not same or not all_items[i][k] == all_items[j][k]:
                            same = False
                    rhs = all_items[i] + (all_items[j][len(all_items[j]) - 1], )
                    if same:
                        lhs = tuple(i for i in itemset if i not in rhs)
                        key = (lhs, rhs)
                        if lhs:
                            c[key] = support_counts[itemset] / support_counts[lhs]
            f, c = c, {}
            for key, value in list(f.items()):
                if value < min_conf:
                    del f[key]
            result.update(f)
    if verbose:
        print(f'Min confidence: {min_conf}, Total rules: {len(result)}, Total rules pruned: {(2 ** len(itemset)) - 2 - len(result)}')
        for key, value in list(result.items()):
            print(f' {key[0]} -> {key[1]}, confidence: {value}')
    return result

supp_counts = Apriori(data, 3, False)
result = Generate_rules(supp_counts, ('Butter', 'Flour', 'Milk'), 0)

Min confidence: 0, Total rules: 6, Total rules pruned: 0
 ('Flour', 'Milk') -> ('Butter',), confidence: 0.9230769230769231
 ('Butter', 'Milk') -> ('Flour',), confidence: 0.7058823529411765
 ('Butter', 'Flour') -> ('Milk',), confidence: 0.6666666666666666
 ('Milk',) -> ('Butter', 'Flour'), confidence: 0.41379310344827586
 ('Flour',) -> ('Butter', 'Milk'), confidence: 0.5714285714285714
 ('Butter',) -> ('Flour', 'Milk'), confidence: 0.5


In [61]:
# I want to generate all rules with that confidence
supp_counts = Apriori(data, 3, False)
results = [{} for i in range(len(supp_counts))]
index = 0
for i in supp_counts:
    results[index] = Generate_rules(supp_counts, i, 0.75, False)
    index += 1
print(f'Min confidence: {0.75}')
for i in results:
    for key, value in list(i.items()):
        print(f' {key[0]} -> {key[1]}, confidence: {value}')    

Min confidence: 0.75
 ('Flour',) -> ('Butter',), confidence: 0.8571428571428571
 ('Butter',) -> ('Flour',), confidence: 0.75
 ('Eggs',) -> ('Milk',), confidence: 0.7647058823529411
 ('Bacon', 'Flour') -> ('Eggs',), confidence: 1.0
 ('Bacon', 'Milk') -> ('Eggs',), confidence: 0.8
 ('Cereal', 'Flour') -> ('Butter',), confidence: 0.8
 ('Eggs', 'Flour') -> ('Butter',), confidence: 0.8181818181818182
 ('Butter', 'Eggs') -> ('Flour',), confidence: 1.0
 ('Butter', 'Eggs') -> ('Milk',), confidence: 0.8888888888888888
 ('Flour', 'Milk') -> ('Butter',), confidence: 0.9230769230769231
 ('Cereal', 'Eggs') -> ('Milk',), confidence: 1.0
 ('Eggs', 'Flour', 'Milk') -> ('Butter',), confidence: 1.0
 ('Butter', 'Eggs', 'Milk') -> ('Flour',), confidence: 1.0
 ('Butter', 'Eggs', 'Flour') -> ('Milk',), confidence: 0.8888888888888888
 ('Butter', 'Eggs') -> ('Flour', 'Milk'), confidence: 0.8888888888888888


#### C. What do you observe or conclude from this activity? Why is this relevant and/or
#### important?

It's interesting that the antimonitone principle works for both frequent itemset generation and rule generation. Pruning low confidence rules is very important for actually finding rules because of the massive number that exist for any given data set.

# Evaluate the Rules

#### A. Find a Rule (if any) with positively associated Lift (> 1)

In [65]:
supp_counts = Apriori(data, 0, False)
results = [{} for i in range(len(supp_counts))]
index = 0
for i in supp_counts:
    results[index] = Generate_rules(supp_counts, i, 0, False)
    index += 1
for i in results:
    for key, value in list(i.items()):
        lift = value / (supp_counts[key[1]] / len(data))
        if lift > 1:
            print(f' {key[0]} -> {key[1]}, lift: {lift}')    

 ('Eggs',) -> ('Bacon',), lift: 1.176470588235294
 ('Bacon',) -> ('Eggs',), lift: 1.1764705882352942
 ('Eggs',) -> ('Butter',), lift: 1.1029411764705883
 ('Butter',) -> ('Eggs',), lift: 1.102941176470588
 ('Flour',) -> ('Butter',), lift: 1.7857142857142856
 ('Butter',) -> ('Flour',), lift: 1.7857142857142858
 ('Milk',) -> ('Butter',), lift: 1.221264367816092
 ('Butter',) -> ('Milk',), lift: 1.2212643678160922
 ('Flour',) -> ('Eggs',), lift: 1.5406162464985995
 ('Eggs',) -> ('Flour',), lift: 1.5406162464985995
 ('Milk',) -> ('Eggs',), lift: 1.3184584178498986
 ('Eggs',) -> ('Milk',), lift: 1.3184584178498986
 ('Milk',) -> ('Flour',), lift: 1.0673234811165846
 ('Flour',) -> ('Milk',), lift: 1.0673234811165846
 ('Bacon', 'Butter') -> ('Cereal',), lift: 1.25
 ('Cereal',) -> ('Bacon', 'Butter'), lift: 1.25
 ('Bacon', 'Butter') -> ('Eggs',), lift: 2.941176470588235
 ('Eggs',) -> ('Bacon', 'Butter'), lift: 2.941176470588235
 ('Bacon', 'Flour') -> ('Butter',), lift: 1.0416666666666667
 ('Bacon

#### B. Find a Rule (if any) with negatively associated Lift (< 1).

In [66]:
supp_counts = Apriori(data, 0, False)
results = [{} for i in range(len(supp_counts))]
index = 0
for i in supp_counts:
    results[index] = Generate_rules(supp_counts, i, 0, False)
    index += 1
for i in results:
    for key, value in list(i.items()):
        lift = value / (supp_counts[key[1]] / len(data))
        if lift < 1:
            print(f' {key[0]} -> {key[1]}, lift: {lift}')  

 ('Butter',) -> ('Bacon',), lift: 0.20833333333333331
 ('Bacon',) -> ('Butter',), lift: 0.20833333333333334
 ('Flour',) -> ('Bacon',), lift: 0.47619047619047616
 ('Bacon',) -> ('Flour',), lift: 0.4761904761904762
 ('Milk',) -> ('Bacon',), lift: 0.4310344827586207
 ('Bacon',) -> ('Milk',), lift: 0.4310344827586207
 ('Cereal',) -> ('Butter',), lift: 0.8333333333333334
 ('Butter',) -> ('Cereal',), lift: 0.8333333333333333
 ('Eggs',) -> ('Cereal',), lift: 0.4411764705882353
 ('Cereal',) -> ('Eggs',), lift: 0.4411764705882352
 ('Flour',) -> ('Cereal',), lift: 0.5952380952380951
 ('Cereal',) -> ('Flour',), lift: 0.5952380952380952
 ('Milk',) -> ('Cereal',), lift: 0.9482758620689654
 ('Cereal',) -> ('Milk',), lift: 0.9482758620689656
 ('Butter', 'Cereal') -> ('Bacon',), lift: 0.3125
 ('Bacon', 'Cereal') -> ('Butter',), lift: 0.2604166666666667
 ('Butter',) -> ('Bacon', 'Cereal'), lift: 0.26041666666666663
 ('Bacon',) -> ('Butter', 'Cereal'), lift: 0.3125
 ('Butter', 'Eggs') -> ('Bacon',), lif

#### C. Suppose we now added 1000 transactions to the dataset, and these new records
#### contained none of the six items we have been considering. Would the Lift change, and
#### does that mean Lift is not a good choice for a measure in that case?

The lift would increase a lot due to larger data. Probably not the best measure.

#### D. What do you observe or conclude from this activity? Why is this relevant and/or
#### important?

Lift depends heavily on the prior P(Y) ditribution. Good thing to note for future use of the measure.


# Evaluate Support Distribution Characteristics

#### A. Compute the measure of cross-support r(X) for the ItemSet X = {Butter,Flour,Milk}.

In [67]:
supp_counts = Apriori(data, 3, False)
item_set = ('Butter', 'Flour', 'Milk')
supps = [supp_counts[(i, )] for i in item_set]
print(f'Cross support: {min(supps) / max(supps)}')

Cross support: 0.7241379310344828


#### B. Compute the hconf(X).

In [68]:
print(f'hconf: {supp_counts[item_set] / max(supps)}')

hconf: 0.41379310344827586


#### C. Is the ItemSet X a hyperclique, for the given threshold hc = 50%?

In [69]:
threshold = 0.5
print("Yes") if supp_counts[item_set] / max(supps) > threshold else print("No")

No


#### D. What do you observe or conclude from this activity? Why is this relevant and/or
#### important?

hconf is a good metric to use for pruning that would allow us to high confidence rule with low support. It will be useful in the future for growing rule sets of high confidence that don't exclude rare items but with a process that's still computationally efficient.

# Takeaways

This is my first experience with finding association rules in a data set. I'll definitly use the process in the future to help discover interactions among categorical model features during exploratory data analysis. I'm excited to run the Apriori algorithm on a larger data set. I have a better understanding of the antimonitone property.